In [2]:
import minsearch

/workspaces/llm-workshop/01-intro/minsearch.py:9: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [3]:
import json


In [ ]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

Bad pipe message: %s [b'0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nHost: localhost:41625\r\nUs', b'-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.']
Bad pipe message: %s [b'0.0 Safari/537.36 OPR/118.0.0.0\r\nAccept-Encodin']
Bad pipe message: %s [b' gzip, deflate, br, zstd\r\nAccept-Language: pl-PL,pl;q=0.9,en-US;q=0.8,en;q=0.7\r\nCache-Control: max-age']
Bad pipe message: %s [b'\r\nReferer: https://shiny-goggles-xrwpp4pxx97cv4j5.github.dev']
Bad pipe message: %s [b'\nX-Request-ID: de87eba384c6f2e642485e51c4589d2', b'\nX-Real-IP: 83.168.77.30\r\nX-Forwarded-Port: 443', b'X-Forwarded-']
Bad pipe message: %s [b'heme: https\r\nX-Original-URI: /\r\nX-Scheme: https\r\nsec-fetch-site: same-site\r\nsec-fe', b'h-mode: navigate\r\nsec-fetch-dest: document\r\nsec-ch-ua: "Not(A:Brand";v="99", "Opera";v="118", "Chromium";v="133"\r\ns', b'-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "Windows"\r\npriority:

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []


In [6]:
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
len(docs_raw)

3

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [10]:
#keywords = SELECT * WHERE course = 'data-engineering-zoomcamp';



In [11]:
q = 'the course has already started, can I still enroll?'


In [12]:
index.fit(documents)

In [13]:
from openai import OpenAI


In [50]:
OPENAI_API_KEY=""
client = OpenAI(api_key=OPENAI_API_KEY)


In [28]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": q}]
)

In [29]:
response

ChatCompletion(id='chatcmpl-BhAZO4u70Q1VdqHIVoIZMT8mAAolb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether you can still enroll in a course that has already started depends on the specific policies of the institution or program offering the course. Here are a few steps you can take:\n\n1. **Check the Enrollment Policy**: Look for information about late enrollment on the program's website or course catalog.\n  \n2. **Contact the Instructor or Administrator**: Reach out directly to the course instructor or the program coordinator to inquire if there is still a possibility to enroll.\n\n3. **Consider Auditing**: Some programs allow students to audit courses after they have started, which means you can attend without earning credit.\n\n4. **Look for Similar Courses**: If enrollment is closed, consider looking for other courses that may have similar content and are still open for enrollment.\n\n5. **Understand the Implications**:

In [30]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [31]:
search(q)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [32]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [33]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [34]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [35]:
rag(query)

'To run Kafka, you can execute the following command in the terminal within your project directory:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your jar file.'

In [36]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. Even if you don't register, you're eligible to submit the homework. However, keep in mind that there are deadlines for turning in the final projects, so it's important not to leave everything to the last minute."

## Elastic search

In [38]:
from elasticsearch import Elasticsearch

In [39]:
es_client = Elasticsearch('http://localhost:9200') 

In [41]:
es_client.info()

ObjectApiResponse({'name': 'c0d475b78e7e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'j0kfnCfIRE2xK1FzMkRa9A', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [40]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [42]:
from tqdm.auto import tqdm


In [44]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [45]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [46]:
query = 'I just disovered the course. Can I still join it?'


In [ ]:
#question is 3 times more important
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [48]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [49]:
rag(query)

'Yes, you can still join the course even after the start date. You are eligible to submit the homeworks without registering. However, be mindful of the deadlines for the final projects, so make sure not to leave everything until the last minute.'